# Functions for HCM CNN

## Padding Function

In [1]:
import numpy as np
import pandas as pd
import math
import random

from scipy import ndimage
from scipy import stats
import sklearn as skl
from sklearn import model_selection as m_s
from skimage.transform import rescale
import tensorflow as tf

In [2]:
# Function to properly pad images
# Input: Patient, current Y_Length, max y_length in Y direction, desired x and y resolution
# Output: Resized and padded image
def pad(image,d_x_res,d_y_res,C_Length,max_length):
    num_frames = image.shape[2]
    p = d_y_res*(C_Length/max_length)
    c_y_res = image.shape[0]
    scale = p/c_y_res
    resized_image = np.zeros((d_x_res,d_y_res,num_frames))
    for j in range(num_frames):
        frame = image[:,:,j]
        resized_frame = rescale(frame,scale)
        pad_row = (256-resized_frame.shape[0])/2
        pad_col = (256-resized_frame.shape[1])/2
        padded_frame = np.pad(resized_frame,((math.floor(pad_row),math.ceil(pad_row)),
                                             (math.floor(pad_col),math.ceil(pad_col))),'constant')
        resized_image[:,:,j] = padded_frame
    return resized_image

## Frame Selection Function

In [3]:
# Function to take images with 50 frames and reduce to 30 frames
# Input: 50 frame image in the form of a 3D np.array
# Output: 30 frame image in the form of a 3D np.array

def frame_selection(image):
    reduced_frames = np.empty((image.shape[0],image.shape[1],30))
    np.random.seed(10)
    odd_indices = np.array([1 + 2*k for k in range(25)])
    even_indices = np.array([2*k for k in range(26)])
    random_indices = np.random.choice(even_indices,5,replace = False)
    all_indices = np.sort(np.concatenate((odd_indices,random_indices)))
    for i, index in enumerate(all_indices):
        reduced_frames[:,:,i] = image[:,:,index]
    return reduced_frames

## Add Dimension Function

In [2]:
# Function to add 4th dimension to image
def add_dim(image, label):
    """Process image by adding a channel."""
    processed_image = tf.expand_dims(image, axis=3)
    return processed_image, label

In [1]:
def standardize(x):
    mu_image = []
    mu = []
    skew = []
    for i in range(training_df.shape[0]):
        image_vec = np.ravel(training_df.Image[i])
        # skew.append(stats.skew(image_vec))
        percentiles = np.percentile(image_vec,[0,99])
        pixels = image_vec[(image_vec>=percentiles[0]) & (image_vec<= percentiles[1])]
        mu_image.append(np.mean(pixels))
        standard = 512*((pixels - percentiles[0])/(percentiles[1] - percentiles[0]))
        # mode2 = standard[standard > np.mean(standard)]
        mu.append(np.mean(standard))